<a href="https://colab.research.google.com/github/llealgt/RL_Workshop_Guatemala/blob/master/Notebooks/Lecture2_Exploration_Exploitation_MAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploración y Explotación en Multi-Armed Bandits

## Material y Background
Basado no solo en el curso de David Silver, también en material de Hado Van Hasselt y el libro de Sutton y Barto



*   https://hadovanhasselt.files.wordpress.com/2016/01/xx2.pdf
*   http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/XX.pdf
*   Sutton y Barto capítulo 2






## Recapitulando

*   Reinforcement learning es la ciencia de "aprender a tomar desicines"
*   Un agente puede aprender una o más de las siguientes opciones:
  *   Política
  *   Función de evaluación
  *   Modelo
*   El problema general involucra tomar en cuenta el tiempo y consecuencias a futuro
*   Las desiciones afectan la recompenza, el conocimiento interno(estado del agente) y el estado del ambiente

![RL_cycle](https://www.unite.ai/wp-content/uploads/2019/10/Rl_agent.png)


*Fuente:* [Daniel Nelson](https://www.unite.ai/what-is-reinforcement-learning/)


## En Esta Sesión
Versión simplificada del problema de RL


*   Múltiples acciones, pero solo un estado.
*   Las decisiones no afectan el estado del ambiente. (acciones pasadas no afectan el futuro)
*   Formalmente: la distribución de la variable $R_{t}$ dado $A_{t}$ no depende del tiempo.
*   Objetivo : optimizar la recompensa inmediata en un juego repetitivo.
*   La historia no incluye observaciones (secuencia de acciones y recompensas)
> $H_{t}=A_{1},R_{1},A_{2},R_{2},...,A_{t},R_{t}$ 





## Ejemplo de la Rata 

![rata_ejemplo1](https://github.com/llealgt/RL_Workshop_Guatemala/blob/master/Assets/lect2_rat1_sample.png?raw=true)


*Fuente:* [**UCL**](https://hadovanhasselt.files.wordpress.com/2016/01/xx2.pdf)


## Ejemplo de la Rata 

![rata_ejemplo1](https://github.com/llealgt/RL_Workshop_Guatemala/blob/master/Assets/lect2_rat2_sample.png?raw=true)


*Fuente:* [UCL](https://hadovanhasselt.files.wordpress.com/2016/01/xx2.pdf)


## El Dilema de Explorar vs Explotar



*   La toma de desiciones involucra una elección fundamental:
  *   **Explotar** : tomar la mejor desición dada la información disponible actualmente.
  *   **Explorar** : obtener mas información/incrementar el conocimiento actual
*   La mejor estrategia a largo plazo puede requerir sacrificios a corto plazo.
*   Se necesita ambas : obtener mas información para tomar las mejores desiciones





## Ejemplos

TODO: 


*   agregar ejemplos que ya están en sesión 1
*   Buscar ejemplos interactivos o animaciones (o "amarrarlo" con los ejemplos interactivos de sesión 1)



## Distintos Enfoques/Principios para Exploración



*   Exploración aleatoria
  *   Explorar acciones aleatorias (ε-greedy)
  *   Con probabilidad ε evaluar una acción al azar.
*   Inicialización optimista
  *   Asumir lo mejor hasta que se demuestre lo contrario.
*   Optimismo ante la incerteza
  *   Preferir acciones con valores inciertos.
*   Probabilística
  *   Seleccionar acciones de acuerdo a la probabilidad de ser la mejor.
*   Estado de la información
  *   Considerar la información del agente parte del estado.
  *   Vista a futuro(lookahead) para ver como la información  ayuda con la recompensa.







## Multi-Armed Bandits
Simplificación del problema de RL.



*   Un MAB es una tupla (A,R)
*   A es un conjunto conocido de m acciones (o "brazos")
*   R es una distribución de probabilidad desconocida sobre las recompensas.
*   En cada momento t el agente selecciona una accion a 
*   El ambiente genera una recompensa r
*   El objetivo es maximizar la recompensa 



![Atari Game Agent and Environment](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTg6W2PLH8geQ_yZDQ_rzQN1hRw3a_8rxJPT8EcyVnlD1a4E8JO)


*Fuente:* [Microsoft Research](https://slivkins.com/work/bandits-svc/)


TODO: 
- buscar un juego interactivo o animación en linea de MAB (o ver si es factible 
hacer uno sencillo con Python)
- Usar notación y formatting de Latex


# Valor de las Acciones

Función de las acciones.



*   El valor de la acción "a" es la recompensa promedio para "a".
>   $q(a) = E[R|A=a]$ 
*   Debemos aproximar o estimar $q(a)$ siendo la forma mas sencilla el promedio de las muestras de las recompensas.
>   $Q_{t}(a) = \frac{\sum_{n=1}^{t}R_{n}I(A_{n}=a)}{\sum_{n=1}^{t}I(A_{n}=a)}$

  Donde $I(True) = 1$ y $I(False) = 0$

Dicho de otra forma: **$Q(a)$** es una aproximación de **$q(a)$** obtenida a través de promediar las recompensas obtenidas al seleccionar la acción **$a$**

TODO: terminar la diapositiva y arreglar formato y notación latex




In [0]:
"""
  Nota: el siguiente código no busca ser óptimo ni usar buenas prácticas de 
  programación. Busca  ilustrar los conceptos y ser fácil de entender.
  TODO: agregar mas comentarios y descripción
"""

class Agent:

  def __init__(self,action_list = []):
    self.action_list = action_list
    self.Q = dict()
    self.action_reward_history = dict()

    for action in action_list:
      self.action_reward_history[action] = []


  def update_Q(self):
    for action in self.action_list:
      self.Q[action] = sum(self.action_reward_history[action])/len(self.action_reward_history[action])

  def observe_action_reward(self,action,reward):
    self.action_reward_history[action].append(reward)

  def print_estimated_q(self):
    for action in self.action_list:
      print("--------")
      print("Estimado de acción {} es {}".format(action,self.Q[action]))

    print("--------")

In [0]:
"""
  Nota: en el siguiente ejemplo el agente no elige acciones y el entorno no provee las recompensas
  - Nosotros elegimos manualmente las acciones.
  - Nosotros damos recompensas arbitrarias.
"""

agent = Agent([1,2,3])

# elegir una vez cada acción y observar la recompensa obtenida
agent.observe_action_reward(1,2)
agent.observe_action_reward(2,-1)
agent.observe_action_reward(3,2)

# actualizar el estimado de q(a) llamado Q(a) y mostrar el estimado
agent.update_Q()
agent.print_estimated_q()

# elegir la segunda acción, observar la recompensa , actualizar el estimado de q(a) y mostrarlo
agent.observe_action_reward(2,1)
agent.update_Q()
agent.print_estimated_q()

--------
Estimado de acción 1 es 2.0
--------
Estimado de acción 2 es -1.0
--------
Estimado de acción 3 es 2.0
--------
--------
Estimado de acción 1 es 2.0
--------
Estimado de acción 2 es 0.0
--------
Estimado de acción 3 es 2.0
--------


## Valor de las Acciones (actualización incremental)

En el caso anterior requerimos guardar la historia de recompensas 

*   Podemos obtener una expresión para una actualización incremental.
  *   Requiere menos memoria
  *   Menor tiempo de computo 
  > $Q_{t}(a_{t}) = Q_{t-1}(a_{t}) + α_{t}(R_{t} - Q_{t-1}(a_{t}))$



*   Es posible(y lo haremos) utilizar otros tamaños de paso $α$(similar al learning rate de aprendizaje supervisado)
  *   Por ejemplo un valor consante de $α$ llevaría a algo llamado "tracking" (contrario a promedio).
      *   Similar a promedio ponderado dando mayor importancia a las muestras mas recientes

TODO: terminar de agregar en latex la definición formal y agregar alguna referencia al promedio incremental.







In [0]:
"""
  Nota: el siguiente código no busca ser óptimo ni usar buenas prácticas de 
  programación. Busca  ilustrar los conceptos y ser fácil de entender.
  TODO: agregar mas comentarios y descripción
"""

class Agent:

  def __init__(self,action_list = []):
    self.action_list = action_list
    self.Q = {a:0 for a in action_list}
    self.N = {a:0 for a in action_list}

  def observe_action_reward(self,action,reward):
    self.N[action] = self.N[action] + 1
    alpha = 1/self.N[action]
    self.Q[action] = self.Q[action] +  alpha*(reward - self.Q[action])
    #TODO: podemos tener 2 versiones de esta celda, una que requiera a los participantes definir los calculos

  def print_estimated_q(self):
    for action in self.action_list:
      print("--------")
      print("Estimado de acción {} es {}".format(action,self.Q[action]))

    print("--------")

In [0]:
"""
  Nota: en el siguiente ejemplo el agente no elige acciones y el entorno no provee las recompensas
  - Nosotros elegimos manualmente las acciones.
  - Nosotros damos recompensas arbitrarias.
"""

agent = Agent([1,2,3])

# elegir una vez cada acción y observar la recompensa obtenida
agent.observe_action_reward(1,2)
agent.observe_action_reward(2,-1)
agent.observe_action_reward(3,2)

# actualizar el estimado de q(a) llamado Q(a) y mostrar el estimado
agent.print_estimated_q()

# elegir la segunda acción, observar la recompensa , actualizar el estimado de q(a) y mostrarlo
agent.observe_action_reward(2,1)
agent.print_estimated_q()

--------
Estimado de acción 1 es 2.0
--------
Estimado de acción 2 es -1.0
--------
Estimado de acción 3 es 2.0
--------
--------
Estimado de acción 1 es 2.0
--------
Estimado de acción 2 es 0.0
--------
Estimado de acción 3 es 2.0
--------


## Ejemplo de la Rata

![rata_ejemplo3](https://github.com/llealgt/RL_Workshop_Guatemala/blob/master/Assets/lect2_rat3_sample.png?raw=true)


*Fuente:* [**UCL**](https://hadovanhasselt.files.wordpress.com/2016/01/xx2.pdf)

> $$\begin{align}
  \ Q_{3}(palanca) = 0  \\
  \ Q_{3}(boton)   = -1 \\
  \end{align}$$


In [0]:
agent = Agent(["palanca","boton"])
rewards_dict = {"queso":1,"shock":-1}

# elegir una vez cada acción y observar la recompensa obtenida
agent.observe_action_reward("boton",rewards_dict["shock"]) #t1
agent.observe_action_reward("palanca",rewards_dict["queso"]) #t2
agent.observe_action_reward("palanca",rewards_dict["shock"]) #t3

# actualizar el estimado de q(a) llamado Q(a) y mostrar el estimado
agent.print_estimated_q()

--------
Estimado de acción palanca es 0.0
--------
Estimado de acción boton es -1.0
--------



![rata_ejemplo3](https://github.com/llealgt/RL_Workshop_Guatemala/blob/master/Assets/lect2_rat4_sample.png?raw=true)


*Fuente:* [**UCL**](https://hadovanhasselt.files.wordpress.com/2016/01/xx2.pdf)






*   Dada la historia entonces:

> $\begin{align}
\ Q_{6}(palanca)  =  -0.6\\
\ Q_{6}(boton)    = -1.0\\
\end{align}$

*   Cuando debería dejar de ser codiciosa ?

TODO: ver como alinear a la izquierda en latex


In [0]:
agent = Agent(["palanca","boton"])
rewards_dict = {"queso":1,"shock":-1}

# elegir una vez cada acción y observar la recompensa obtenida
agent.observe_action_reward("boton",rewards_dict["shock"]) #t1
agent.observe_action_reward("palanca",rewards_dict["queso"]) #t2
agent.observe_action_reward("palanca",rewards_dict["shock"]) #t3
agent.observe_action_reward("palanca",rewards_dict["shock"]) #t4
agent.observe_action_reward("palanca",rewards_dict["shock"]) #t5
agent.observe_action_reward("palanca",rewards_dict["shock"]) #t6

# actualizar el estimado de q(a) llamado Q(a) y mostrar el estimado
agent.print_estimated_q()

--------
Estimado de acción palanca es -0.6
--------
Estimado de acción boton es -1.0
--------


## Valor Optimo y Acción Optima



*   La acción óptima  $a^*$ es la acción que maximiza a la función de valor.
  > $a^* =\underset{a\in A}{\operatorname{argmax}}\ q(a)$
*   El valor óptimo $v_{*}$ es el valor de la acción óptima. 
  > $v_{*} = q(a^*) = \underset{a\in A}{\operatorname{max}}\ q(a) =\underset{a\in A}{\operatorname{max}}\ E[R|A=a]$



In [0]:
def calc_optimos(q={}):
  """
    Dado un diccionario q representando a la función de valor devuelve:
      la acción optima
      el valor óptimo
  """

  accion_optima = list(q.keys())[0]
  valor_optimo = q[accion_optima]

  for accion in q.keys():
    valor = q[accion]

    if valor > valor_optimo:
      valor_optimo = valor
      accion_optima = accion

  return accion_optima,valor_optimo

Por ejemplo:

In [0]:
accion_optima,valor_optimo = calc_optimos({1:1,2:2,3:3,0:8,10:5})

print("accion y valor optimos: {},{}".format(accion_optima,valor_optimo))

accion y valor optimos: 0,8


## Regret (arrepentimiento)

El arrepentimiento es la oportunidad perdida en el momento t.

>  $l_{t} = v_{*} - q(A_{t})$

Ejemplo: si la acción optima es tomar el bus con un valor de 8 y en el momento
"t" tomo un taxi con un valor de 5 mi arrepentimiento es de **3**.

(perdí la oportunidad de tener 3 unidades mas)


**Nota**
*   El arrepentimiento es una herramienta para analizar diferentes algoritmos
  *   No es parte de ningún algoritmo.
  *   El agente no puede observarlo u obtener muestras del arrepentimiento real.
  *   Depende de conocer el valor óptimo y esto es precisamente algo que el agente desconoce y busca aprender.
  *   Herramienta para el desarrollador.







In [0]:
def calc_arrepentimiento(valor_optimo,q,accion_t):
  l = valor_optimo - q[accion_t]

  return l

Arrepentimiento:


In [0]:
q = {1:1,2:2,3:3,0:8,10:5}
accion_t = 1
accion_optima,valor_optimo = calc_optimos(q)
arrepentimiento = calc_arrepentimiento(valor_optimo,q,accion_t)

print("Arrepentimiento de acción {} es {}".format(accion_t,arrepentimiento))

Arrepentimiento de acción 1 es 7


**Arrepentimiento total**: es la oportunidad perdida total desde el tiempo i=1 hasta t

> $L_{t} = \sum_{i=1}^{t} l_{t} = \sum_{i=1}^{t} (v_{*} - q(a_{i}))$



*   **Objetivo:** Balancear la exploración y explotación de manera que se minimice  el arrepentimiento total. 
*   Maximizar recompensa acumulada ≡ minimizar arrepentimiento total.



**$A_{t}$**

**$R_{t}$**